# Simons' Algorithm

In [1]:
""" This module implements Simons periodicity algorithm"""
from sympy.physics.quantum.qapply import qapply
from sympy.physics.quantum.qubit import Qubit, matrix_to_qubit, \
     measure_partial, IntQubit
from sympy import Symbol, preorder_traversal, Xor, Or, And, \
    satisfiable, Not, simplify
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.represent import represent
from oracle import oracle
from util.util import hn

In [2]:
def find_req_on_f(c, ys):
    """
    Find requirements on f
    :param int c: number c
    :param tuple ys: range of values
    :returns collection of [(val, req) ... ]
    example:
    c= 0b101 ys=[0,1,...,7]
    returns  [(0, 5),(1, 4),...,(7, 2)]
    """
    xs = [y ^ c for y in ys]
    return zip(ys, xs)

In [3]:
def find_c(constraints):
    """
    Find c given the constraints
    :param tuple collection constraints:
    :returns int c
    example
    [(0, 0, 0), (0, 1, 0), (1, 0, 1), (1, 1, 1)]
    we create the equivalent expression
    ~b & ~(a ^ c) & (a | b | c) & ~(a ^ b ^ c)
    and solve for it
    """
    bit_num = len(constraints[0])

    # limit to 16 bits
    assert bit_num < 16
    for i in range(bit_num):
        Symbol(chr(ord('a') + i))

    # create the equivalent expression NOT( c_1 XOR c_2 XOR ... c_n-1)
    expressions = []
    for elem in constraints:
        expressions.append(Not(Xor(*[elem[i] *
                                     Symbol(chr(ord('a') + i))
                                     for i in range(bit_num)])))

    # add final expression (c_1 OR c_2 OR ... c_n-1)
    # as we do not want a solution where all Cs are zero
    expressions.append(Or(*[Symbol(chr(ord('a') + i))
                            for i in range(bit_num)]))

    # put them in one
    fe = And(*expressions)
    # solve it
    models = satisfiable(fe, all_models=True)

    # find the equivalent number C (a collection of them)
    ret = []
    for model in models:
        model = sorted(model.items(), key=lambda item: item[0].name)
        r = 0
        n = bit_num - 1
        for d in model:
            r = r + d[1] * 2 ** n
            n = n - 1
        ret.append(r)
    # sort the results for ease of comparison
    return sorted(ret)

## The problem:
Consider `f {0,1}^n  -> f {0, 1}^n` that is not known

we are told that exists `c = c0c1c2...cn-1` such that for all string x, y belonging to `{0,1}^n`
we have `f(x) == f(y)`  if and only if `x = y XOR c`

## The Solution:
```
                     +-----------------+
  |x>  |0>-/n--H*n---|                 |--/n--H*n---M |x>
                     |       U_f       |
  |y>  |0>-/n--------|                 |--/n--------  |y XOR f(x)>
                     +-----------------+
```

In [4]:
def simon(f, n):
    """
    Run Simon algorithm and return |x>*|y XOR f(x)>
    :param func f: oracle function
    :param int n: string length

    """
    # apply H gate to both inputs
    x = qapply(hn(n) * Qubit('0' * n))
    print(f"|x>: {x}")
    y = Qubit('0' * n)
    print(f"|y>: {y}")
    xy = TensorProduct(x, y)
    print(f"|xy>: {xy}")
    xy = matrix_to_qubit(represent(xy))
    print(f"|xy>: {xy}")

    # apply oracle
    state = oracle(x, y, f)

    # apply H*m to the top bits
    state = simplify(qapply(hn(n, start=3) * state))
    print(f'State: {state}')

    # measure the first 3 bits
    measures = measure_partial(state, range(n))
    print(f'Measures: {measures}')

    constraints = []
    for m in measures:
        m_contr = []
        for arg in preorder_traversal(m[0]):
            if isinstance(arg, Qubit):
                m_contr.append(arg.qubit_values[:3])
        constraints.append(m_contr)

    # find c
    c = find_c(constraints[0])
    print(f'c: {c[0] :03b}')

    #  Find requirements on f(x)
    req = find_req_on_f(c[0], range(0, 2 ** 3))
    print(f'Req: {list(req)}')

    return state, measures, c, constraints[0], req

## Tests

Find requirements on f given binary string `0b101`

In [5]:
def test_0():
    r = find_req_on_f(0b101, range(0, 2 ** 3))
    truth = [(0, 5), (1, 4), (2, 7), (3, 6), (4, 1), (5, 0), (6, 3), (7, 2)]

    for k in zip(r, truth):
        assert k[0] == k[1]


test_0()

Find requirements on f given binary string `0b011`

In [6]:
def test_1():
    r = find_req_on_f(0b011, range(0, 2 ** 3))

    truth = [(0, 3), (1, 2), (2, 1), (3, 0), (4, 7), (5, 6), (6, 5), (7, 4)]
    for k in zip(r, truth):
        assert k[0] == k[1]


test_1()

Test finc C

In [7]:
def test_2():
    constraints = ((1, 1, 1), (1, 0, 1), (0, 1, 0))
    r = find_c(constraints)

    truth = [0b101]
    assert truth == r


test_2()

Test finc C

In [8]:
def test_3():
    constraints = ((1, 0, 1, 0, 1, 1, 0),
                   (0, 0, 1, 0, 0, 0, 1),
                   (1, 1, 0, 0, 1, 0, 1),
                   (0, 0, 1, 1, 0, 1, 1),
                   (0, 1, 0, 1, 0, 0, 1),
                   (0, 0, 1, 1, 0, 1, 0),
                   (0, 1, 1, 0, 1, 1, 1))
    r = find_c(constraints)

    truth = [0b1101010]
    assert truth == r


test_3()

Test Simon's algorithm

In [ ]:
def test_4():
    def f(x, *args):
        x = IntQubit(Qubit(*x)).as_int()
        match x:
            case 0b000:
                return 0b100
            case 0b101:
                return 0b100
            case 0b001:
                return 0b001
            case 0b100:
                return 0b001
            case 0b010:
                return 0b101
            case 0b111:
                return 0b101
            case 0b011:
                return 0b111
            case 0b110:
                return 0b111

    n = 3
    truth_state = 1 / 4 * (TensorProduct(Qubit('000'), Qubit('100')
                           + Qubit('001') + Qubit('101') + Qubit('111')) +
                           TensorProduct(Qubit('010'), Qubit('100')
                           + Qubit('001') - Qubit('101') - Qubit('111')) +
                           TensorProduct(Qubit('101'), Qubit('100')
                           - Qubit('001') + Qubit('101') - Qubit('111')) +
                           TensorProduct(Qubit('111'), Qubit('100')
                           - Qubit('001') - Qubit('101') + Qubit('111')))
    truth_constraints = [(0, 0, 0), (0, 1, 0), (1, 0, 1), (1, 1, 1)]
    truth_c = 0b101
    state, measures, c, constraints, req = simon(f, n)
    assert constraints == truth_constraints

    truth_state = matrix_to_qubit(represent(truth_state))
    assert simplify(truth_state - state) == 0
    assert c[0] == truth_c


test_4()

|x>: sqrt(2)*|000>/4 + sqrt(2)*|001>/4 + sqrt(2)*|010>/4 + sqrt(2)*|011>/4 + sqrt(2)*|100>/4 + sqrt(2)*|101>/4 + sqrt(2)*|110>/4 + sqrt(2)*|111>/4
|y>: |000>
|xy>: (sqrt(2)*|000>/4 + sqrt(2)*|001>/4 + sqrt(2)*|010>/4 + sqrt(2)*|011>/4 + sqrt(2)*|100>/4 + sqrt(2)*|101>/4 + sqrt(2)*|110>/4 + sqrt(2)*|111>/4)x|000>
|xy>: sqrt(2)*|000000>/4 + sqrt(2)*|001000>/4 + sqrt(2)*|010000>/4 + sqrt(2)*|011000>/4 + sqrt(2)*|100000>/4 + sqrt(2)*|101000>/4 + sqrt(2)*|110000>/4 + sqrt(2)*|111000>/4
State: (|000001> + |000100> + |000101> + |000111> + |010001> + |010100> - |010101> - |010111> - |101001> + |101100> + |101101> - |101111> - |111001> + |111100> - |111101> + |111111>)/4
Measures: [(|000001>/2 + |010001>/2 - |101001>/2 - |111001>/2, 1/4), (|000100>/2 + |010100>/2 + |101100>/2 + |111100>/2, 1/4), (|000101>/2 - |010101>/2 + |101101>/2 - |111101>/2, 1/4), (|000111>/2 - |010111>/2 - |101111>/2 + |111111>/2, 1/4)]
c: 101
Req: [(0, 5), (1, 4), (2, 7), (3, 6), (4, 1), (5, 0), (6, 3), (7, 2)]


In [ ]:
def test_5():
    def f(x, *args):
        x = IntQubit(Qubit(*x)).as_int()
        match x:
            case 0b000:
                return 0b000
            case 0b001:
                return 0b100
            case 0b010:
                return 0b100
            case 0b011:
                return 0b000
            case 0b100:
                return 0b010
            case 0b101:
                return 0b110
            case 0b110:
                return 0b110
            case 0b111:
                return 0b010

    n = 3
    truth_state = 1 / 4 * (TensorProduct(Qubit('000'), Qubit('000')
                           + Qubit('010') + Qubit('100') + Qubit('110')) +
                           TensorProduct(Qubit('011'), Qubit('000')
                           + Qubit('010') - Qubit('100') - Qubit('110')) +
                           TensorProduct(Qubit('100'), Qubit('000')
                           - Qubit('010') + Qubit('100') - Qubit('110')) +
                           TensorProduct(Qubit('111'), Qubit('000')
                           - Qubit('010') - Qubit('100') + Qubit('110')))
    truth_constraints = [(0, 0, 0), (0, 1, 1), (1, 0, 0), (1, 1, 1)]
    truth_c = 0b011
    state, measures, c, constraints, req = simon(f, n)

    assert constraints == truth_constraints
    truth_state = matrix_to_qubit(represent(truth_state))
    assert simplify(truth_state - state) == 0
    assert c[0] == truth_c


test_5()

|x>: sqrt(2)*|000>/4 + sqrt(2)*|001>/4 + sqrt(2)*|010>/4 + sqrt(2)*|011>/4 + sqrt(2)*|100>/4 + sqrt(2)*|101>/4 + sqrt(2)*|110>/4 + sqrt(2)*|111>/4
|y>: |000>
|xy>: (sqrt(2)*|000>/4 + sqrt(2)*|001>/4 + sqrt(2)*|010>/4 + sqrt(2)*|011>/4 + sqrt(2)*|100>/4 + sqrt(2)*|101>/4 + sqrt(2)*|110>/4 + sqrt(2)*|111>/4)x|000>
|xy>: sqrt(2)*|000000>/4 + sqrt(2)*|001000>/4 + sqrt(2)*|010000>/4 + sqrt(2)*|011000>/4 + sqrt(2)*|100000>/4 + sqrt(2)*|101000>/4 + sqrt(2)*|110000>/4 + sqrt(2)*|111000>/4
State: (|000000> + |000010> + |000100> + |000110> + |011000> + |011010> - |011100> - |011110> + |100000> - |100010> + |100100> - |100110> + |111000> - |111010> - |111100> + |111110>)/4
Measures: [(|000000>/2 + |011000>/2 + |100000>/2 + |111000>/2, 1/4), (|000010>/2 + |011010>/2 - |100010>/2 - |111010>/2, 1/4), (|000100>/2 - |011100>/2 + |100100>/2 - |111100>/2, 1/4), (|000110>/2 - |011110>/2 - |100110>/2 + |111110>/2, 1/4)]
c: 011
Req: [(0, 3), (1, 2), (2, 1), (3, 0), (4, 7), (5, 6), (6, 5), (7, 4)]
